<br>
<img align="center" src="imagenes/logo.png"  width="200" height="141">
<font size=36><center> Machine Learning con Python </center> </font>
<br>

<h1 align='center'> Modulo IV: Machine Learning </h1>
<h2 align='center'>  Optimización de Hiperparámetros  </h2> 

---

# Aspectos Generales

## Introducción

Uno de los principales objetivos del proceso de aprendizaje automático es mejorar la calificación del rendimiento de los modelos, basándose en los patrones de datos y en las pruebas observadas. Para lograr este objetivo, casi todos los algoritmos de aprendizaje automático tienen un conjunto específico de parámetros que necesita estimar a partir del conjunto de datos, lo que maximizará la puntuación de rendimiento. La mejor manera de elegir un buen hiperparámetro es a través de la prueba y el error de todas las combinaciones posibles de valores de los parámetros. Scikit-learn proporciona las funciones **GridSearchCV** y **RandomSearchCV** para facilitar un enfoque automático y reproducible para el ajuste de hiperparámetros.

## GridSearch

Para un modelo determinado, puede definir un conjunto de valores de parámetros que le gustaría probar. Entonces, utilizando la función GridSearchCV de scikit-learn, se construyen modelos para todas las posibles combinaciones posibles de una lista preestablecida de valores de hiperparámetros que usted proporciona, y la mejor combinación se elige en base a la puntuación de validación cruzada. 

Hay dos desventajas asociadas a GridSearchCV:

1. **Costoso desde el punto de vista computacional:** Es obvio que con más valores de los parámetros el GridSearch será mayor el costo computacional. Considere un ejemplo en el que tiene 5 parámetros y suponga que quiere probar 5 valores para cada parámetros, lo que dará lugar a 5**5 = 3125 combinaciones; además multiplique esto por el número de pliegues de validación cruzada que se utilicen, es decir, si k-fold es 5, entonces 3125 * 5 = 15625 ajustes del modelo.

2. **No se trata de parámetros óptimos perfectos, sino casi óptimos:** La búsqueda en la cuadrícula se centra en los puntos fijos que usted proporciona para los parámetros numéricos, por lo que hay una gran posibilidad de de perder el punto óptimo que se encuentra entre los puntos fijos. Por ejemplo, suponga que quiere probar los puntos fijos para 'n_estimadores': [100, 250, 500, 750, 1000] para un modelo de árbol de decisión y existe la posibilidad de que el punto óptimo se encuentre los dos puntos fijos; sin embargo, GridSearch no está diseñado para buscar entre los puntos fijos.

## RandomSearch

Como su nombre indica, el algoritmo **RandomSearch** prueba combinaciones aleatorias de un rango de valores de parámetros dados. Los parámetros numéricos pueden especificarse como un rango (a diferencia de los valores fijos de GridSearch). Puede controlar el número de iteraciones de búsquedas aleatorias que desea realizar. Es conocido por encontrar una muy buena combinación en mucho menos tiempo comparado con GridSearch; sin embargo, tiene que elegir cuidadosamente el rango de parámetros y el número de iteraciones de búsqueda aleatoria, ya que puede perderse la mejor combinación de parámetros con menos iteraciones o rangos más pequeños.

# Ejemplo

Probemos GridSearchCV y RandomForest en un problema de clasificación, y veamos cuál modelo encuentra los valores óptimos de los parámetros.

### Importamos las librerías

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from scipy.stats import randint as sp_randint

seed = np.random.seed(2017) 

### Dataset

Este conjunto de datos proviene de los Institutos Nacionales de Diabetes y Enfermedades Digestivas y Renales. El propósito de este conjunto de datos es diagnosticar si un paciente es diabético o no, sobre la base de ciertas medidas de diagnóstico en el conjunto de datos. La selección de estas instancias de una base de datos más grande estuvo sujeta a varias restricciones. Todos los pacientes son mujeres de herencia indígena de Pima, de al menos 21 años.

In [2]:
df = pd.read_csv('datos/diabetes.csv')

In [3]:
df.head(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [4]:
X = df.drop(columns='Outcome') # variables independientes
y = df['Outcome'].values # variable dependiente

### Normalizamos los datos

In [5]:
X = StandardScaler().fit_transform(X)

### Separamos los datos para el entrenamiento y la validación

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=seed)

In [7]:
kfold = StratifiedKFold(n_splits = 5, random_state=seed)

### Creamos el modelo

In [9]:
modelo_rf = RandomForestClassifier(random_state=seed).fit(X_train, y_train)
rf_parametros = {
 'n_estimators': [100, 250, 500, 750, 1000],
 'criterion': ['gini', 'entropy'],
 'max_features': [None, 'auto', 'sqrt', 'log2'],
 'max_depth': [1, 3, 5, 7, 9]
}

### Busqueda de los hiperparámetros óptimos usando GridSearch

In [10]:
grid = GridSearchCV(modelo_rf, rf_parametros, scoring='roc_auc', cv=kfold, verbose=10, n_jobs=-1)

In [34]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 3, 5, 7, 9],
                         'max_features': [None, 'auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 250, 500, 750, 1000]},
             scoring='roc_auc', verbose=10)

### Resultados

In [35]:
print('Mejores parámetros: ', grid.best_params_)

Mejores parámetros:  {'criterion': 'entropy', 'max_depth': 7, 'max_features': 'log2', 'n_estimators': 500}


In [36]:
resultados = cross_val_score(grid.best_estimator_, X_train,y_train, cv=kfold)

In [43]:
print("Accuracy - Train CV: ", resultados.mean())
print("Accuracy - Train : ", accuracy_score(grid.best_estimator_.predict(X_train), y_train))

Accuracy - Train CV:  0.7522672204915196
Accuracy - Train :  0.7878787878787878


In [44]:
print("Accuracy - Test : ", accuracy_score(grid.best_estimator_.predict(X_test), y_test))

Accuracy - Test :  0.7878787878787878


### Busqueda de los hiperparámetros óptimos usando RandomSearch

In [48]:
param_dist = {'n_estimators':sp_randint(100,1000),
 'criterion': ['gini', 'entropy'],
 'max_features': [None, 'auto', 'sqrt', 'log2'],
 'max_depth': [None, 1, 3, 5, 7, 9]
 }

In [52]:
n_iter_search = 20
random_search = RandomizedSearchCV(modelo_rf, param_distributions=param_dist, cv=kfold, n_iter=n_iter_search, verbose=10, n_jobs=-1, random_state=seed)

In [53]:
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   estimator=RandomForestClassifier(), n_iter=20, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [None, 1, 3, 5, 7, 9],
                                        'max_features': [None, 'auto', 'sqrt',
                                                         'log2'],
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000019C051AFC10>},
                   verbose=10)

### Resultados

In [54]:
print('Mejores Parametros: ', random_search.best_params_)

Mejores Parametros:  {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'auto', 'n_estimators': 293}


In [55]:
resultados = cross_val_score(random_search.best_estimator_, X_train,y_train, cv=kfold)

In [57]:
print("Accuracy - Train CV: ", resultados.mean())
print("Accuracy - Train : ", accuracy_score(random_search.best_estimator_.predict(X_train), y_train))
print("Accuracy - Test : ", accuracy_score(random_search.best_estimator_.predict(X_test), y_test))

Accuracy - Train CV:  0.7541709934233298
Accuracy - Train :  0.845437616387337
Accuracy - Test :  0.7835497835497836
